# Notebook 2: Docking of LSD1  to ORY-1001 and GSK2879552 KDM1A Inhibitors

---

## Step 1: Define the Docking Region

Before we can dock our ligands to LSD1, we need to specify the region of the protein where docking will take place. This docking region, or **ligand box**, confines the space where AutoDock Vina will explore ligand binding conformations, helping to focus on the primary binding site and save computational time.

To define the docking region, we will:
1. **Load the Protein Structure**: Use MDAnalysis to load the prepared protein structure file.
2. **Identify the Bound Ligand**: By locating the primary ligand within the protein structure, we can establish the binding pocket.
3. **Calculate the Center and Dimensions of the Docking Box**: We will define the center of the box around the ligand using its center of geometry. Then, we'll expand the box dimensions slightly to allow for ligand flexibility during docking.

### Code Breakdown:
- **Pocket Center**: Using the center of the ligand geometry, this point will act as the docking box center.
- **Box Dimensions**: By calculating the dimensions based on the ligand’s position, we add a 5 Å buffer to allow for flexibility and potential interactions within the binding site.

This defined region will guide the docking software in evaluating potential binding poses of our ligands.




In [1]:
import MDAnalysis as mda  # Import MDAnalysis for protein structure handling
import numpy as np  # Import numpy for array operations

# Load the original structure with both protein and ligand
combined_structure_path = "protein_structures/2UXX.pdb"  # File with both protein and ligand
combined_structure = mda.Universe(combined_structure_path)

# Select the ligand by resname (e.g., FAJ) for defining the docking region
ligand = combined_structure.select_atoms("resname FAJ")

# Calculate the center of geometry of the ligand to define the docking pocket center
pocket_center = ligand.center_of_geometry()

# Calculate the box dimensions by taking the min and max coordinates, adding a 5 Å buffer
ligand_min = ligand.positions.min(axis=0) - 5  # Min with buffer
ligand_max = ligand.positions.max(axis=0) + 5  # Max with buffer
box_dimensions = ligand_max - ligand_min  # Final box dimensions

# Convert pocket center and box dimensions to lists for compatibility with Vina
pocket_center = pocket_center.tolist()
box_dimensions = box_dimensions.tolist()

# Print the pocket center and box dimensions for reference
print(f"Docking Pocket Center: {pocket_center}")
print(f"Box Dimensions: {box_dimensions}")


Docking Pocket Center: [66.82566682119219, 64.15730146377805, 31.455619024851966]
Box Dimensions: [17.53499984741211, 35.0829963684082, 22.4950008392334]


##### Step 1.1 Compare to Center of Mass
Run the center_of_mass.py script which reads the residues for FAJ from an FAJ_atoms.txt file and calculates the center of mass

In [2]:
 %run center_of_mass.py

Center of Mass (x, y, z): (66.82566666666666, 64.15730158730157, 31.455619047619034)


In [4]:
!pip install vina

  Using cached vina-1.2.5.tar.gz (89 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for vina (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [113 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      Version found 1.2.5 (from __init__.py)
      running bdist_wheel
      running build
      running build_ext
      Boost library is not installed in this conda environment.
      Boost library location was automatically guessed at /usr/include.
      - include_dirs: ['/home/secondbook5/miniconda3/envs/docking/include/python3.11', '/usr/include', 'src/lib']
      - library_dirs: ['/usr/lib/x86_64-linux-gnu']
      - swig options: ['-c++', '-small', '-naturalvar', '-fastdispatch', '-shadow', '-py3', '-I/home/secondbook5/miniconda3/envs/docking/include/python3.11', '-I/usr/include',

In [3]:
from vina import Vina  # Import Vina for docking calculations
import os  # Import os for directory handling

# Define the base name of the ligand files without extension
ligands = ["ory1001", "gsk2879552", "faj"]
pdb_id = "2UXX"
# Generate full paths for ligand PDBQT files dynamically
ligand_pdbqt_paths = [f"pdbqt/{ligand}.pdbqt" for ligand in ligands]

# Define path for the receptor PDBQT file
protein_pdbqt_path = f"pdbqt/protein_{pdb_id}_cleaned.pdbqt"

# Ensure output directory exists
output_dir = "docking_results"
os.makedirs(output_dir, exist_ok=True)

# Initialize Vina with the Vina scoring function
v = Vina(sf_name="vina")
v.set_receptor(protein_pdbqt_path)

# Use pre-defined pocket center and box dimensions from the previous code chunk
v.compute_vina_maps(center=pocket_center, box_size=box_dimensions)

# Print confirmation and ligand paths for reference
print("AutoDock Vina setup complete with defined receptor and docking box.")
print("Ligand PDBQT paths:", ligand_pdbqt_paths)


ModuleNotFoundError: No module named 'vina'